<font color='red' size=5><b> Software Requirements: The below scripts are tested in Python version 3.6.1 and Seaborn version 0.7.1</b> </font><br>   
 
# <font color='blue'>Hackathon: Pro Kabaddi League</font> <br>
# <font color='blue'> Important Instructions: Execute this second </font>
# <font color='blue'> _______________ </font>
## <font color='blue'> Setup Working directory, copy files, execute scripts </font>
<font color='blue'> <br>
1) Copy input CSV files into your home/working directory <br>
2) To perform web scraping, please copy the provided Chrome Driver into the working directory
2) Copy this ipython (ipynb) notebook into your home/working directory <br>
3) Ensure that the CSV files and this ipython notebook (ipynb) are in the home/working directory <br>
4) Execute below scripts by sequence order </font> <br>

# Webscraping from www.prokabaddi.com for Team and Player information using Selenium Driver and Beautiful Soup

In [32]:
from bs4 import BeautifulSoup
import selenium
from selenium  import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import requests
import pandas as pd

def invoke_main(url,path):
    driver = webdriver.Chrome()
    player_list=[]
    driver.get(url)
    if (path =='player'):
        wait(driver, 600).until(EC.presence_of_element_located((By.XPATH, '//*[@id="player_Btn"]')))
# or just wait for a second for browser(driver) to change
        driver.implicitly_wait(1)
    elif (path == 'team'):
         wait(driver, 600).until(EC.presence_of_element_located((By.XPATH, '//*[@id="team_Btn"]')))
# or just wait for a second for browser(driver) to change
         driver.implicitly_wait(1)
    else:
        print("path not found")
    while True:
        try:
            driver.implicitly_wait(1)
        #wait(driver, 600).until(EC.presence_of_element_located((By.XPATH, '//*[@id="load_more"]')))
            wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="load_more"]'))).click()
            soup = BeautifulSoup(driver.page_source,'lxml')
        except selenium.common.exceptions.TimeoutException:
            break
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    driver.close();
    return soup

def invoke_class_func(soup,class_name):
    if (class_name =='sipk-lb-detailBlock sipk-lb-team'):
        player_profiles = soup.findAll('a', attrs={'class': 'sipk-lb-detailBlock sipk-lb-team', 'href': True})
        list_rec=[]
        for profile in player_profiles:
            #print(profile.find('a')['href'])
            #print(profile.find('a').contents[0])
            profile_name = profile['href']
            
            profile_name = profile_name.split("/",1)[1]
            profile_name = profile_name.replace("teams/","")
            profile_name = profile_name.split('-profile',1)[0]

            #profile_name = profile_name.replace("\n","")
            #profile_name = profile_name.strip()
            list_rec.append(profile_name)
    else:
        name_list = soup.findAll('div', attrs={'class': class_name})
        list_rec=[]
        for lst in name_list:
            rec1 = lst.find('span').text
            rec1 = rec1.replace("\n","")
            rec1 = rec1.strip()
            list_rec.append(rec1)
        
    return list_rec
    
    

In [5]:
#Working well on this
url_array = ["https://www.prokabaddi.com/stats/0-102-total-points-statistics","https://www.prokabaddi.com/stats/0-21-successful-raids-statistics","https://www.prokabaddi.com/stats/0-22-raid-points-statistics","https://www.prokabaddi.com/stats/0-23-successful-tackles-statistics","https://www.prokabaddi.com/stats/0-103-tackle-points-statistics","https://www.prokabaddi.com/stats/0-138-avg-raid-points-statistics","https://www.prokabaddi.com/stats/0-139-avg-tackle-points-statistics","https://www.prokabaddi.com/stats/0-132-do-or-die-raid-points-statistics","https://www.prokabaddi.com/stats/0-104-super-raids-statistics","https://www.prokabaddi.com/stats/0-28-super-tackles-statistics","https://www.prokabaddi.com/stats/0-100-super-10s-statistics","https://www.prokabaddi.com/stats/0-101-high-5s-statistics"]
path = "player"
    
def getplayerInfo(url_array_new, season_num):
    for url in url_array_new:
        season = str(season_num)
        soup = invoke_main(url,path)
        name_list = invoke_class_func(soup,'sipk-lb-playerName')
        matches_played = invoke_class_func(soup,'sipk-lb-detailBlock sipk-lb-matchedPlayed')
        raid_points = invoke_class_func(soup,'sipk-lb-detailBlock sipk-lb-raidPoints')
        teams = invoke_class_func(soup,'sipk-lb-detailBlock sipk-lb-team')
        #name_list = soup.findAll('div', attrs={'class': 'sipk-lb-playerName'})
        str1 = url
        str1 = str1.replace('https://www.prokabaddi.com/stats/','')
        str2 = str1.split('-')
        str_final = str2[2] + '-' + str2[3]
        file_str = str2[2] + '-' + str2[3] + '-' + str2[4]
        file_name = 'prokabaddi' + '_' + 'player' + '_' + file_str +  '_' + season + '.csv'
        df = "df" + "_" + str_final
        df = pd.DataFrame(name_list,columns =['player_name'])
        df['Season'] = season
        df['matches_played'] = matches_played
        df['team'] = teams
        df[str_final] = raid_points
        #write the result to the CSV file
        file_name = 'prokabaddi' + '_' + str_final +  '_' + season + '.csv'
        df.to_csv(file_name,index=False)

for j in range(0,8):
    if (j==5):
        
        url_array_new = [j.replace("/0-","/8-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
    elif (j==6):
        
        url_array_new = [j.replace("/0-","/26-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
    elif (j==7):
        
        url_array_new = [j.replace("/0-","/49-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
        
    else:
        str1 = "/" +str(j)+"-"
        url_array_new = [j.replace("/0-",str1) for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)

0
1
2
3
4
5
6
7


In [34]:
import os
url_array = ["https://www.prokabaddi.com/stats/0-133-total-points-conceded-statistics",
             "https://www.prokabaddi.com/stats/0-140-avg-points-scored-statistics",
             "https://www.prokabaddi.com/stats/0-13-successful-raids-statistics",
             "https://www.prokabaddi.com/stats/0-97-raid-points-statistics",
             "https://www.prokabaddi.com/stats/0-98-avg-raid-points-statistics",
             "https://www.prokabaddi.com/stats/0-15-successful-tackles-statistics",
             "https://www.prokabaddi.com/stats/0-95-tackle-points-statistics",
             "https://www.prokabaddi.com/stats/0-99-avg-tackle-points-statistics",
             "https://www.prokabaddi.com/stats/0-134-super-raid-statistics",
             "https://www.prokabaddi.com/stats/0-20-super-tackles-statistics",
             "https://www.prokabaddi.com/stats/0-135-do-or-die-raid-points-statistics",
             "https://www.prokabaddi.com/stats/0-136-all-outs-inflicted-statistics",
             "https://www.prokabaddi.com/stats/0-137-all-outs-conceded-statistics"]
path = "team"
    
def getplayerInfo(url_array_new, season_num):
    for url in url_array_new:
            season = str(season_num)
            str1 = url
            str1 = str1.replace('https://www.prokabaddi.com/stats/','')
            str2 = str1.split('-')
            
            str_final = str2[2] + '-' + str2[3] 
            file_str = str_final + '-' + str2[4]
            file_name = 'prokabaddi' + '_' + 'team' + '_' + file_str +  '_' + season + '.csv'
            isfile = os.path.isfile(file_name)
            if(~isfile):
                
                soup1 = invoke_main(url,path)
                if (soup1):
                    name_list = invoke_class_func(soup1,'sipk-lb-playerName')
                    matches_played = invoke_class_func(soup1,'sipk-lb-detailBlock sipk-lb-matchedPlayed')
                    raid_points = invoke_class_func(soup1,'sipk-lb-detailBlock sipk-lb-raidPoints')
        #name_list = soup.findAll('div', attrs={'class': 'sipk-lb-playerName'})
            
                    df = "df" + "_" + str_final
                    df = pd.DataFrame(name_list,columns =['team_name'])
                    df['Season'] = season
                    df['matches_played'] = matches_played
                    df[str_final] = raid_points
            #write the result to the CSV file

                    df.to_csv(file_name,index=False)

for j in range(0,8):
    if (j==5):
        
        url_array_new = [j.replace("/0-","/8-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
    elif (j==6):
        
        url_array_new = [j.replace("/0-","/26-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
    elif (j==7):
        
        url_array_new = [j.replace("/0-","/49-") for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)
        
    else:
        str1 = "/" +str(j)+"-"
        url_array_new = [j.replace("/0-",str1) for j in url_array]
        getplayerInfo(url_array_new,j)
        print(j)

0
1
2
3
4
5
6
7


## End of Web Scraping for Team and Player data. Check the working directory for the CSV output files